In [17]:
import tensorflow as tf
import numpy as np

VGG_16_weights_dict = np.load('/Users/vijay/Downloads/vgg16_weights.npz')


############################################

def get_placeholders_for_train_label_data(no_of_classes):
    
    x       = tf.placeholder(tf.float32, shape = [None, 224 * 224 * 3], name = 'x_rs')
    x_rs    = tf.reshape(x, shape = [-1, 224, 224, 3])
    
    y       = tf.placeholder(tf.float32, shape = [None, no_of_classes])
    y_rs    = tf.reshape(y, shape = [-1, no_of_classes])
    dropout = tf.placeholder(tf.float32, name = 'dropout')
    
    return x_rs, y_rs, dropout



############################################

def get_weights_or_bias_from_VGG(wts_or_bias_name):
    return VGG_16_weights_dict[wts_or_bias_name]




############################################

def get_weights_or_bias_as_variables(wts_or_bias_name, var_name):
    wts_or_bias    = get_weights_or_bias_from_VGG(wts_or_bias_name)
    wt_or_bias_init = tf.constant_initializer(values = wts_or_bias, dtype = tf.float32)
    wt_or_bias_var  = tf.get_variable(shape = np.shape(wts_or_bias, initializer = wt_or_bias_init, name = var_name))
    
    return wt_or_bias_var



############################################

def fetch_and_reshape_fc_weights(fc_wts_name, rs_shape):
    fc_wts      = VGG_16_weights_dict[fc_wts_name]
    fc_wts      = fc_wts.reshape(rs_shape)
    fc_wts_init = tf.constant_initializer(values = fc_wts, dtype = tf.float32)
    fc_wts_var  = tf.get_variable(shape = rs_shape, initializer = fc_wts_init, name = 'fc_wts')
    
    return fc_wts_var



############################################

def conv_layer(data, flt_name, bias_name, res_name):

    with tf.variable_scope(res_name):
        conv_wt       = get_weights_or_bias_as_variables(flt_name, 'conv_wt')
        conv_bias     = get_weights_or_bias_as_variables(bias_name, 'conv_bias')
        
        conv_res      = tf.nn.conv2d(data, filter = conv_wt, strides = [1, 1, 1, 1], padding = 'SAME')
        conv_res_bias = tf.nn.bias_add(conv_res, conv_bias)
        conv_res_relu = tf.nn.relu(conv_res_bias)

    return conv_res_relu



############################################

def pooling_layer(data, pool_name):
    with tf.variable_scope(pool_name):
        return tf.nn.max_pool(value = data, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

    

############################################

def fc_layer(data, dropout, fc_wts_name = None, fc_bias_name = None):
    
    if fc_wts_name is not None and fc_bias_name is not None:
        fc_wt_shape = []
        
        if fc_wts_name == 'fc6_W':
            fc_wt_shape = [7, 7, 512, 4096]
        if fc_wts_name == 'fc7_W':
            fc_wt_shape = [1, 1, 4096, 4096]
        
        with tf.variable_scope(fc_wts_name):
            fc_wts = fetch_and_reshape_fc_weights(fc_wts_name, fc_wt_shape)            
            fc_bias = get_weights_or_bias_as_variables(fc_bias_name, 'fc_b')
            
            fc_conv = tf.nn.conv2d(data, filter = fc_wts, strides = [1, 1, 1, 1], padding = 'SAME')
            fc_conv_bias = tf.nn.bias_add(fc_conv, fc_bias)
            fc_conv_relu = tf.nn.relu(fc_conv_bias)
            
        return tf.nn.droput(fc_conv_relu, dropout)
            

        
############################################

def skip_connection(from_data, con_name, flt_val):
    with tf.variable_scope(con_name):
        skp_wt = tf.Variable(tf.truncated_normal(flt_val, stddev = 0.2, seed = 23, dtype = tf.float32), name = 'skip_wt')
        skp_conv = tf.nn.conv2d(from_data, skp_wt, [1, 1, 1, 1], padding = 'SAME')
        
        skp_bias = tf.Variable(tf.constant(0.1, shape = [flt_val[-1]], dtype = tf.float32), name = 'skip_bias')
        skip_res = tf.bias_add(skp_conv, skp_bias)
        return skip_res

    

############################################

def upsample(from_data, op_shape, no_of_channels, name, factor):
    flt_size = 2 * factor - factor % 2
    strides = [1, factor, factor, 1]
    with tf.variable_scope(name):
        filter_shape = [flt_size, flt_size, no_of_channels, no_of_channels]
        weights = get_bilinear_weights(filter_shape, factor)
        deconv = tf.nn.conv2d_transpose(from_data, weights, op_shape, strides = strides, padding = 'SAME')
        upsmp_bias = tf.Variable(tf.constant(0.1, shape = [op_shape[-]]), name = 'upsmp_bias')
        
        deconv_bias = tf.bias_add(deconv, upsmp_bias)
        
    return deconv_bias




############################################

def get_bilinear_weights(filter_shape, factor):
    flt_size = filter_shape[0]
    if flt_size % 2 == 0:
        centre = factor - 1
    else:
        centre = factor - 0.5
        
    bilinear = np.zeros([flt_size, flt_size])
    weigths = np.zeros(filter_shape)
    for row in range(filter_shape[0]):
        for column in range(filter_shape[1]):
            cell_val = (1 - abs((row - centre) / factor)) * (1 - abs((column - centre) / factor))
            bilinear[row, column] = cell_val
    
    for val in range(filter_shape[2]):
        weights[:, :, val, val] = bilinear
    
    wt_init = tf.constant_initializer(value = weights, dtype = tf.float32)
    bilinear_weights = tf.Variable(initializer = wt_init, name = 'blnr_wt', shape = filter_shape)
    
    return bilinear_weights




In [30]:
np.shape(VGG_16_weights_dict['conv5_3_W'])

(3, 3, 512, 512)